# Let's play with Polars

## 1. Resolve dependencies

In [1]:
import pl from "npm:nodejs-polars";
import { display } from "https://deno.land/x/display@v0.1.1/mod.ts";

## 2. Fetch Data sources

In [2]:
const response_atih = await fetch( "https://raw.githubusercontent.com/decoderleco/deces_europe/main/data/csv/Atih.csv", );
let data_atih = await response_atih.text();

const response_dept_regions =  await fetch( "https://raw.githubusercontent.com/decoderleco/deces_europe/main/data/csv/departements-region.csv", );
let data_DR = await response_dept_regions.text();

## 2.bis Clean data

_the data headers are quoted in the CSV Files: we remove any " character from all frames data_

In [3]:
const data_atih_cleaned = data_atih.replace(/"/g, "");
const data_DR_cleaned = data_DR.replace(/"/g, "");

## 3. Load data as Data Frames

In [4]:

const df_atih = pl.readCSV(data_atih_cleaned, { sep: "," });
const df_DR = pl.readCSV(data_DR_cleaned, { sep: "," });

## 4. Display first records of each dataset

In [5]:
df_DR.head(5)

num_dep,dep_name,region_name
01,Ain,Auvergne-Rhône-Alpes
02,Aisne,Hauts-de-France
03,Allier,Auvergne-Rhône-Alpes
04,Alpes-de-Haute-Provence,Provence-Alpes-Côte d'Azur
05,Hautes-Alpes,Provence-Alpes-Côte d'Azur


In [6]:
df_atih.head(5)

dep,annee,tauxPatients,nombre_patients
01,2017,185.73,114500
02,2017,198.12,106386
03,2017,193.71,70599
04,2017,200.37,34032
05,2017,201.41,29840


## 5. Select where

In [7]:
// df_DR.filter(pl.col("foo").lt(3).and(pl.col("ham").eq(pl.lit("a"))))
// df_atih.filter(pl.col("tauxPatients").lt(200).and(pl.col("annee").eq("2017")))
df_atih.filter(pl.col("tauxPatients").lt(200)).sort({by: "tauxPatients"}).head(7)
// df_atih.select(pl.col('dep'))

dep,annee,tauxPatients,nombre_patients
976,2017,62.82,13846
976,2018,65.47,13788
976,2020,66.67,12959
976,2019,67.76,14333
976,2021,69.41,13714
977,2017,132.05,1082
978,2020,136.44,4048


In [8]:
df_atih.filter(pl.col("tauxPatients").lt(200)).sort({by: "annee"}).head(7)

dep,annee,tauxPatients,nombre_patients
01,2017,185.73,114500
02,2017,198.12,106386
03,2017,193.71,70599
07,2017,187.79,63466
08,2017,199.64,55796
09,2017,185.57,30002
10,2017,181.56,56962


In [9]:
df_atih.filter(pl.col("tauxPatients").lt(140)).filter(pl.col("annee").gt(2019))

dep,annee,tauxPatients,nombre_patients
976,2020,66.67,12959
978,2020,136.44,4048
976,2021,69.41,13714


In [10]:
df_atih.filter(
    pl.col("tauxPatients").gt(200)
    .and(pl.col("annee").eq(2017))
    .and(pl.col("nombre_patients").lt(50000))
              )

dep,annee,tauxPatients,nombre_patients
04,2017,200.37,34032
05,2017,201.41,29840
55,2017,205.79,40105


## 6. GroupBy

In [11]:
df_atih.groupBy("annee").count()

annee,dep_count,tauxPatients_count,nombre_patients_count
2020,104,104,104
2019,104,104,104
2018,104,104,104
2021,104,104,104
2017,104,104,104


## 7. Aggregation functions

In [24]:
df_atih.groupBy("annee").agg(pl.col("nombre_patients").sum().alias("Nombre annuel de patients"))

annee,Nombre annuel de patients
2017,25308661
2019,25698108
2020,23162660
2021,25267786
2018,25512826
